### --- Day 21: Keypad Conundrum ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/21

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 21);
WriteLines(inputLines, maxRows: 3);

Loading puzzle file: Day21.txt
Total lines: 5
Max line length: 4

869A
180A
596A


In [4]:
string[] testInputLines = [
    "029A",
    "980A",
    "179A",
    "456A",
    "379A",
];

Ok, for starters, we know that for a given keypad we can pre-calculate the minimum steps from one key to another.

Things get more complex once we connect the keypads together however, as the position of prior keypads might affect the cost of lower keypads. Eg: if the prior keypad is already in the up position, then going up + right would be cheaper than going right + up.

So I think we need to evaluate the possible combinations.



In [5]:
using Key = char;
using KeySequence = string;

In [6]:
using FromToDict = SCG.Dictionary<(Key from, Key to), SCG.IEnumerable<KeySequence>>;
const Key EMPTY = '.';
static readonly Point Push = (0, 0);

class Keypad
{
    readonly Dictionary<Point, Key> Keys = new();
    readonly Dictionary<Key, Point> Positions = new();
    readonly FromToDict _fromToCache = new();

    public Keypad(string[] inputLines)
    {
        CharGrid grid = new (inputLines);

        foreach (var (point, ch) in grid.Enumerate().Where(pch => pch.ch is not EMPTY))
        {
            Keys[point] = ch;
            Positions[ch] = point;
        }
    }

    /// <summary>
    /// Get the shortest sequences of directions from one key to another
    /// </summary>
    public IEnumerable<KeySequence> GetSequences(Key fromKey, Key toKey)
    {
        if (fromKey == toKey) { return PushOnly; }
        
        var cacheKey = (fromKey, toKey);
        if (_fromToCache.TryGetValue(cacheKey, out var result))
        {
            return result;
        }

        var (from, to) = (Positions[fromKey], Positions[toKey]);
        var diff = to - from;

        var distance = Math.Abs(diff.X) + Math.Abs(diff.Y);
        var horizDir = diff.X < 0 ? Left : Right;
        var vertDir = diff.Y < 0 ? Up : Down;

        var q = from directions in Permutations([horizDir, vertDir], distance)
                where IsValidPath(@from, to, directions)
                select Stringify(directions.Append(Push));

        result = q.ToArray();
        _fromToCache[cacheKey] = result;
        
        return result;
    }

    bool IsValidPath(Point start, Point end, IEnumerable<Point> directions)
    {
        var current = start;
        foreach (var dir in directions)
        {
            current += dir;
            if (!Keys.ContainsKey(current))
            {
                return false;
            }
        }
        return current == end;
    }

    static readonly IEnumerable<KeySequence> PushOnly = ["A"];
    static readonly Dictionary<Point, Key> DirMap = new()
    {
        { Up, '^' },
        { Down, 'v' },
        { Left, '<' },
        { Right, '>' },
        { Push, 'A' }
    };
    static KeySequence Stringify(IEnumerable<Point> dirs) => new(dirs.Select(dir => DirMap[dir]).ToArray());
}

In [7]:
// +---+---+---+
// | 7 | 8 | 9 |
// +---+---+---+
// | 4 | 5 | 6 |
// +---+---+---+
// | 1 | 2 | 3 |
// +---+---+---+
//     | 0 | A |
//     +---+---+

string[] numpad = 
[
    "789",
    "456",
    "123",
    ".0A"
];

//     +---+---+
//     | ^ | A |
// +---+---+---+
// | < | v | > |
// +---+---+---+

string[] dirpad = 
[
    ".^A",
    "<v>"
];

In [8]:
// So for the sequence 029A, we need to find...

// Directions from A (starting point) to 0
// Directions from 0 to 2
// Directions from 2 to 9
// Directions from 9 to A

// Then, given a set of directions...

// We need to map up/down/left/right/push as keys on the new keypad
// Then, for each direction pair, we need to get the sequence
// Find the sum of the sequence, and we're done

In [9]:
IEnumerable<KeySequence> GetWholeSequence(KeySequence sequence, string[] keys)
{
    Keypad np = new(keys);

    return GetPair(sequence, sequence.Length - 1);
    
    IEnumerable<KeySequence> GetPair(KeySequence sequence, int pos)
    {
        var result = pos switch {
            0 => np.GetSequences('A', sequence[0]),
            _ => GetPair(sequence, pos - 1)
                    .SelectMany(gp => np.GetSequences(sequence[pos-1], sequence[pos])
                                            .Select(seq => $"{gp}{seq}"))
        };

        return result;
    } 
}

// In total, there are three shortest possible sequences of button presses on
// this directional keypad that would cause the robot to type 029A: <A^A>^^AvvvA,
// <A^A^>^AvvvA, and <A^A^^>AvvvA.

var getWholeSequenceTest = GetWholeSequence("029A", numpad);
foreach (var seq in getWholeSequenceTest)
{
    Console.WriteLine(seq);
}

<A^A>^^AvvvA
<A^A^>^AvvvA
<A^A^^>AvvvA


In [10]:
int GetMinLength(string combo)
{
    var q = from lvl1 in GetWholeSequence(combo, numpad)
            from lvl2 in GetWholeSequence(lvl1, dirpad)
            from lvl3 in GetWholeSequence(lvl2, dirpad)
            select lvl3.Length;

    return q.Min();
}

// The length of the shortest sequence of button presses ... for 029A, this would be 68.

var testGetMinLength = GetMinLength("029A");
Console.WriteLine(testGetMinLength);

68


In [11]:
int GetComplexity(string combo)
{
    // The complexity of a single code (like 029A) is equal to the result of multiplying these two values:

    // The length of the shortest sequence of button presses...
    var shortestSequence = GetMinLength(combo);
    
    // The numeric part of the code (ignoring leading zeroes); for 029A, this would be 29.
    var numericPart = int.Parse(combo[0..3]);

    return numericPart * shortestSequence;
}

// In the above example, complexity of the five codes can be found by
// calculating 68 * 29, 60 * 980, 68 * 179, 64 * 456, and 64 * 379. Adding these
// together produces 126384.

var testAnswer = testInputLines.Select(GetComplexity).Sum();
Console.WriteLine(testAnswer);

126384


In [12]:
// Find the fewest number of button presses you'll need to perform in order to
// cause the robot in front of the door to type each code. What is the sum of the
// complexities of the five codes on your list?

var part1Answer = inputLines.Select(GetComplexity).Sum();
Console.WriteLine(part1Answer);

248108


In [13]:
// 248108 is correct!
Ensure(248108, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/21

Ok, for part 2, there are now 25 robots! There must be a more general way solve this than we did for part 1...

The challenge I had with this part was that I couldn't convince myself that lower robots were always in a known position, although in retrospect this is indeed the case. Having skimmed some solutions for ideas and taking a fresh look, a few insights helped me towards a solution.

The first insight that helped is to consider the sequence in terms of "indirection levels", where 0 means standing directly in front of a keypad, and 1 means via 1 robot, and 2 via another robot etc.

Consider the string `029A`:

* The cost of entering `029A` at indirection level `0` is just `4`, as we can press each key directly
* The cost of entering `029A` at indirection level `1` means we are standing in front of a robot, that is...
* The cost of entering `029A` at indirection level `1` is equivalent to entering a longer sequence at indirection level `0`

So in general, entering a sequence at indirection level `n` can be re-framed as solving a longer sequence at indirection level `n-1`

But how do we know what this "longer sequence" is? It's the sequence of individial key transitions!

Which brings us to the second insight: for a given indirection level, the cost of transitioning from key `X->Y` is always the same! There's a rough proof:

* Transitioning from `X->Y` at indirection level `0` is `1`, since we are standing in front of it. Therefore `X->Y` is always equal.
* Tarnsitioning from `X->Y` at indirection level `1` means standing in front of a robot, intialised to `A`, and entering the shortest sequence that must necessarily end back at `A`, as this key is the Push action. So since key transitions always start / finish at `A`, a given transition will always have the same cost.

This recursive approach is readily cacheable. Now we have everything we need to solve.

In [15]:
long GetMinCost(KeySequence keySequence, int levels)
{
    Dictionary<(Key from, Key to, int indirectionLevel), long> costCache = new();
    
    return GetMinCost(keySequence, levels, levels);

    long GetMinCost(KeySequence keySequence, int levels, int indirectionLevel)
    {
        if (indirectionLevel == 0)
        {
            // Just push the keys directly!
            return keySequence.Length;
        }
        
        // At this point we are "indirect".

        // Eg for: `029A`, we are solving for A->0, 0->2, 2->9, etc
        var pairs = keySequence.Prepend('A').Zip(keySequence);

        // As explained above, the total cost is the sum of the individual key
        // transitions
        long total = 0;
        foreach (var (a, b) in pairs)
        {
            total += GetPairCost(a, b, indirectionLevel);
        }
        return total;

        long GetPairCost(Key a, Key b, int indirectionLevel)
        {
            var cacheKey = (a, b, indirectionLevel);
            if (costCache.TryGetValue(cacheKey, out var result))
            {
                return result;
            }

            // Time to re-frame:
            //
            // Given this key transition and indirection level, what would be
            // the equivalent sequence we could input at the lower indirection
            // level?

            // There could be multiple sequences; naturally we take the shortest
            // one.

            var keys = indirectionLevel == levels ? numpad : dirpad;
            Keypad keypad = new(keys);

            result = keypad.GetSequences(a, b)
                        .Min(seq => GetMinCost(seq, levels, indirectionLevel-1));
            
            costCache[cacheKey] = result;

            return result;
        }
    }
}

In [16]:
// Re-test part 1 with the new algorithm:

long GetComplexity2(string combo, int levels)
{
    var numericPart = int.Parse(combo[0..3]);
    
    var shortestSequence = GetMinCost(combo, levels);

    return numericPart * shortestSequence;
}

long GetComplexityPt1(string combo) => GetComplexity2(combo, 3);

var testAnswerV2 = testInputLines.Select(GetComplexityPt1).Sum();
Console.WriteLine(testAnswerV2);

var part1AnswerV2 = inputLines.Select(GetComplexityPt1).Sum();
Console.WriteLine(part1AnswerV2);

126384
248108


In [17]:
// Find the fewest number of button presses you'll need to perform in order to
// cause the robot in front of the door to type each code. What is the sum of the
// complexities of the five codes on your list?

var part2Answer = inputLines.Select(line => GetComplexity2(line, 26)).Sum();
Console.WriteLine(part2Answer);

303836969158972


In [18]:
// 303836969158972 is correct
Ensure(303836969158972, part2Answer);